## 1) Introduction :
Le taux de désabonnement des clients se produit lorsque les clients ou les abonnés cessent de faire affaire avec une entreprise ou un service. Une entreprise veut savoir quels sont les clients qui vont se désabonner en examinant certains des attributs importants et en y appliquant le Machine Learning ou le Deep Learning.

## 2) Contexte du projet

Le taux de désabonnement fait référence au moment où un client met fin à sa relation avec une entreprise. Les entreprises en ligne considèrent un client désabonné une fois qu'un certain temps s'est écoulé depuis la dernière interaction du client avec le site ou le service.

Un modèle de désabonnement prédictif est un outil qui définit les étapes de l'attrition d'un client, ou le départ d'un client d'un service ou produit. En exploitant ce modèle de désabonnement en évolution, l'entreprise peut lutter pour cette rétention.


Le code doit bien être structuré, avec des explications sur le choix de l'architecture utilisée dans le modèle.

L'apprenant doit utiliser que les anciens notebooks pour réaliser ce travail (Interdit d'utiliser l'internet)


## 3) SCRIPT

### 1) Import des bibliothèques

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,cross_val_predict

from sklearn.metrics import mean_squared_error,r2_score,confusion_matrix,f1_score,precision_recall_curve,roc_curve,roc_auc_score
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml
import cv2

### 2) Import des données

In [21]:
df = pd.read_csv("./data.csv")
df

,num_ligne,ID_Client,Nom,Score_Credit,Pays,Sex,Age,Tenure,Balance,Num_Produit,il_a_CrCard,Membre_actif,Salaire_estime,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### 3) Prétraitement des données

#### nettoyage

In [22]:
df.isna().sum()

num_ligne         0
ID_Client         0
Nom               0
Score_Credit      0
Pays              0
Sex               0
Age               0
Tenure            0
Balance           0
Num_Produit       0
il_a_CrCard       0
Membre_actif      0
Salaire_estime    0
Exited            0
dtype: int64

In [23]:
df.isna().values.any()

False

#### encodage

In [24]:
# Récupération des index de colonnes catégorielles :
col = df.columns

col_num = df._get_numeric_data().columns
col_num

col_cat = list(set(col)-set(col_num))

# Création d'un système pour encoder plusieurs colones d'un coup et garder un dico avec les objets label encoder
# correspondant afin de pouvoir faire un inverse transform pour repasser en valeur qualitative si on le souhaite.
dico_lab = dict.fromkeys(col_cat, 0)
for col in col_cat:

    dico_lab[col] = LabelEncoder()
    dico_lab[col].fit(df[col])
    
    df[col] = dico_lab[col].transform(df[col])

print(dico_lab)
df.head(5)

{'Sex': LabelEncoder(), 'Pays': LabelEncoder(), 'Nom': LabelEncoder()}


,num_ligne,ID_Client,Nom,Score_Credit,Pays,Sex,Age,Tenure,Balance,Num_Produit,il_a_CrCard,Membre_actif,Salaire_estime,Exited
0,1,15634602,1115,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,1177,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,2040,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,289,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,1822,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# Pour décoder une colonne encodée avec ce système : 
dico_lab['Nom'].inverse_transform(df["Nom"])

array(['Hargrave', 'Hill', 'Onio', ..., 'Liu', 'Sabbatini', 'Walker'],
      dtype=object)

#### Set d'apprentissage et d'essai

In [569]:
df.corr()

,num_ligne,ID_Client,Nom,Score_Credit,Pays,Sex,Age,Tenure,Balance,Num_Produit,il_a_CrCard,Membre_actif,Salaire_estime,Exited
num_ligne,1.000000,0.004202,0.001682,0.005840,-0.010358,0.018196,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
ID_Client,0.004202,1.000000,0.005689,0.005308,0.006516,-0.002641,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
Nom,0.001682,0.005689,1.000000,0.007489,-0.022878,-0.002049,0.005550,-0.017412,0.002657,-0.016460,-0.008993,0.001483,0.011850,-0.010822
Score_Credit,0.005840,0.005308,0.007489,1.000000,0.007888,-0.002857,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Pays,-0.010358,0.006516,-0.022878,0.007888,1.000000,0.004719,0.022812,0.003739,0.069408,0.003972,-0.008523,0.006724,-0.001369,0.035943
Sex,0.018196,-0.002641,-0.002049,-0.002857,0.004719,1.000000,-0.027544,0.014733,0.012087,-0.021859,0.005766,0.022544,-0.008112,-0.106512
Age,0.000783,0.009497,0.005550,-0.003965,0.022812,-0.027544,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,-0.017412,0.000842,0.003739,0.014733,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.002657,0.006268,0.069408,0.012087,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
Num_Produit,0.007246,0.016972,-0.016460,0.012238,0.003972,-0.021859,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820


In [568]:
df.head()

,num_ligne,ID_Client,Nom,Score_Credit,Pays,Sex,Age,Tenure,Balance,Num_Produit,il_a_CrCard,Membre_actif,Salaire_estime,Exited
0,1,15634602,1115,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,1177,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,2040,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,289,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,1822,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [84]:
X = df.iloc[:,1:-1]
X.head()

,ID_Client,Nom,Score_Credit,Pays,Sex,Age,Tenure,Balance,Num_Produit,il_a_CrCard,Membre_actif,Salaire_estime
0,15634602,1115,619,0,0,42,2,0.00,1,1,1,101348.88
1,15647311,1177,608,2,0,41,1,83807.86,1,0,1,112542.58
2,15619304,2040,502,0,0,42,8,159660.80,3,1,0,113931.57
3,15701354,289,699,0,0,39,1,0.00,2,0,0,93826.63
4,15737888,1822,850,2,0,43,2,125510.82,1,1,1,79084.10


In [83]:
y = df.iloc[:,-1:]
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [301]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True)

In [113]:
x_train.loc[1:1,:].shape

(1, 12)

In [276]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling3D, Conv2D
import numpy as np

In [ ]:
x_train2 = x_train.to_numpy()
x_test2 = x_test.to_numpy()

Standardisation des variables :

In [397]:
stand_sc = StandardScaler()
x_train2 = stand_sc.fit_transform(x_train2)
x_test2 = stand_sc.fit_transform(x_test2)

In [398]:
y_train2 = y_train.to_numpy()
y_test2 = y_test.to_numpy()

#### Création et entraînement du modèle de deep learning :

Les données étant unidimensionnelles et n'étant pas des images, je préfère essayer de faire un réseau de neuronne simple (ANN) plutôt qu'un CNN.

In [557]:
# Instanciation du modèle
model = Sequential()

# Couche d'entrée :
model.add(Dense(12, input_dim = 12, activation='relu'))

# Couches intermédiaires :
model.add(Dense(9, activation='relu'))
model.add(Dense(9, activation='relu'))

# Couche de sortie :
model.add(Dense(1, activation='sigmoid'))

# Compilation du modèle
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Entraîenement :
history = model.fit(x_train2,
                    y_train2,
                    epochs=200)

Epoch 1/200
250/250 [==============================] - 2s 2ms/step - loss: 0.5711 - accuracy: 0.7871
Epoch 2/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4482 - accuracy: 0.8039
Epoch 3/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4386 - accuracy: 0.7965
Epoch 4/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4284 - accuracy: 0.8101
Epoch 5/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4114 - accuracy: 0.8213
Epoch 6/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4027 - accuracy: 0.8250
Epoch 7/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3977 - accuracy: 0.8322
Epoch 8/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3885 - accuracy: 0.8374
Epoch 9/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3712 - accuracy: 0.8456
Epoch 10/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3643 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.3098 - accuracy: 0.8738
Epoch 82/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3028 - accuracy: 0.8773
Epoch 83/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3118 - accuracy: 0.8727
Epoch 84/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3176 - accuracy: 0.8721
Epoch 85/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3023 - accuracy: 0.8756
Epoch 86/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3203 - accuracy: 0.8709
Epoch 87/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3123 - accuracy: 0.8733
Epoch 88/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3132 - accuracy: 0.8752
Epoch 89/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3098 - accuracy: 0.8743
Epoch 90/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3087 - accuracy: 

250/250 [==============================] - 0s 1ms/step - loss: 0.2953 - accuracy: 0.8817
Epoch 161/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3107 - accuracy: 0.8749
Epoch 162/200
250/250 [==============================] - 0s 1ms/step - loss: 0.2925 - accuracy: 0.8840
Epoch 163/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3037 - accuracy: 0.8754
Epoch 164/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3083 - accuracy: 0.8761
Epoch 165/200
250/250 [==============================] - 0s 1ms/step - loss: 0.2969 - accuracy: 0.8797
Epoch 166/200
250/250 [==============================] - 0s 1ms/step - loss: 0.2903 - accuracy: 0.8842
Epoch 167/200
250/250 [==============================] - 0s 1ms/step - loss: 0.2913 - accuracy: 0.8815
Epoch 168/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3005 - accuracy: 0.8783
Epoch 169/200
250/250 [==============================] - 0s 1ms/step - loss: 0.3030 - a

In [558]:
predictions = model.predict(x_test2)
predictions

array([[0.9666766 ],
       [0.0489383 ],
       [0.225043  ],
       ...,
       [0.0815953 ],
       [0.02645993],
       [0.09889734]], dtype=float32)

In [559]:
for i in range(0,len(y_test2)):
    print(y_test2[i])
    print(predictions[i])
    print()

[1]
[0.9666766]

[1]
[0.0489383]

[0]
[0.225043]

[0]
[0.4508636]

[0]
[0.10049754]

[0]
[0.12850508]

[0]
[0.86280656]

[0]
[0.10947654]

[0]
[0.00387427]

[0]
[0.00860509]

[0]
[0.17687038]

[0]
[0.1567741]

[0]
[0.08586165]

[0]
[0.06538865]

[0]
[0.00436869]

[0]
[0.00399747]

[0]
[0.00277707]

[0]
[0.3910674]

[0]
[0.07443601]

[1]
[0.57287896]

[0]
[0.01548612]

[1]
[0.9999815]

[0]
[0.00100243]

[0]
[0.02569941]

[1]
[0.17573547]

[1]
[0.9365363]

[0]
[0.01591185]

[0]
[0.37865987]

[1]
[0.27592695]

[0]
[0.09660327]

[0]
[0.2218698]

[1]
[0.9806972]

[0]
[0.10798946]

[0]
[0.05727437]

[0]
[0.03361914]

[0]
[0.06032684]

[0]
[0.01685026]

[0]
[0.29278058]

[0]
[0.5000291]

[0]
[0.04402721]

[0]
[0.00997961]

[0]
[0.00451109]

[1]
[0.27496707]

[0]
[0.01693636]

[0]
[0.17678955]

[0]
[0.08919072]

[1]
[0.9562473]

[0]
[0.4085781]

[1]
[0.523785]

[0]
[0.042907]

[0]
[0.08335432]

[0]
[0.04190618]

[0]
[0.3473131]

[0]
[0.36715004]

[0]
[0.01655737]

[0]
[0.03778002]

[0]
[0.0506

[0]
[0.15726537]

[0]
[0.07740799]

[0]
[0.00433803]

[0]
[0.08034968]

[1]
[0.14602154]

[0]
[0.13318697]

[0]
[0.4139368]

[1]
[0.8014716]

[1]
[0.1260683]

[0]
[0.19115472]

[0]
[0.27709144]

[0]
[0.02471548]

[0]
[0.00437301]

[0]
[0.03258762]

[0]
[0.10527092]

[0]
[0.01211953]

[0]
[0.5724602]

[0]
[0.0159786]

[1]
[0.0872654]

[1]
[0.0416908]

[0]
[0.00294453]

[1]
[0.6348406]

[0]
[0.1640572]

[0]
[0.05048481]

[0]
[0.03081542]

[1]
[0.94501245]

[0]
[0.8387019]

[0]
[0.03106397]

[0]
[0.00422639]

[1]
[0.9414861]

[0]
[0.01778397]

[1]
[0.09219217]

[0]
[0.00724131]

[0]
[0.33679458]

[1]
[0.7679746]

[1]
[0.9887725]

[1]
[0.08392906]

[0]
[0.0119085]

[1]
[0.9857137]

[0]
[0.58498454]

[0]
[0.31160086]

[0]
[0.20295382]

[0]
[0.5914017]

[1]
[0.92201877]

[0]
[0.09693134]

[0]
[0.0577133]

[0]
[0.3745234]

[0]
[0.01744002]

[0]
[0.03931344]

[0]
[0.07513666]

[1]
[0.15735346]

[0]
[0.04967526]

[0]
[0.3261895]

[0]
[0.26588008]

[0]
[0.03394318]

[1]
[0.42622545]

[0]
[0.1552


[0]
[0.04513395]

[0]
[0.07728392]

[0]
[0.18205312]

[0]
[0.3707037]

[0]
[0.19401997]

[1]
[0.9999902]

[0]
[0.102862]

[0]
[0.03712541]

[1]
[0.49352336]

[0]
[0.05026883]

[0]
[0.05312067]

[0]
[0.1269545]

[0]
[0.5608724]

[0]
[0.05508134]

[1]
[0.08301643]

[1]
[0.6602055]

[0]
[0.6760285]

[1]
[0.9569352]

[1]
[0.05324677]

[0]
[0.03087723]

[0]
[0.0207426]

[0]
[0.32532388]

[0]
[0.03640994]

[0]
[0.28161305]

[0]
[0.04505357]

[0]
[0.0278779]

[1]
[0.28099135]

[0]
[0.07203981]

[0]
[0.9323599]

[0]
[0.03105924]

[1]
[0.77711505]

[1]
[0.05561677]

[0]
[0.08697379]

[1]
[0.893875]

[0]
[0.0245643]

[0]
[0.7508937]

[0]
[0.00414249]

[0]
[0.05728418]

[0]
[0.06226116]

[0]
[0.12089649]

[0]
[0.00622788]

[0]
[0.01279309]

[0]
[0.23182464]

[0]
[0.02502638]

[0]
[0.02315155]

[0]
[0.01872525]

[1]
[0.9339247]

[1]
[0.9698089]

[1]
[0.06621069]

[1]
[0.45004758]

[0]
[0.2131649]

[0]
[0.20664716]

[1]
[0.45186466]

[1]
[0.9752262]

[0]
[0.03995797]

[0]
[0.29290247]

[1]
[0.2238

[0]
[0.05989596]

[1]
[0.2373603]

[0]
[0.07762504]

[0]
[0.00860125]

[0]
[0.11607769]

[0]
[0.03914252]

[1]
[0.9949498]

[0]
[0.2825734]

[0]
[0.09785059]

[0]
[0.10810024]

[0]
[0.01517299]

[0]
[0.04743892]

[0]
[0.03090021]

[0]
[0.05557531]

[0]
[0.01564723]

[0]
[0.14899772]

[1]
[0.6947313]

[0]
[0.26328844]

[0]
[0.05575627]

[0]
[0.01251915]

[0]
[0.00885645]

[0]
[0.19674855]

[1]
[0.22034422]

[0]
[0.03850865]

[1]
[0.2630725]

[0]
[0.06959316]

[1]
[0.56809103]

[0]
[0.02835217]

[0]
[0.00512636]

[0]
[0.57275784]

[0]
[0.13184991]

[0]
[0.03229272]

[0]
[0.01188871]

[1]
[0.9999801]

[0]
[0.17912066]

[0]
[0.01192379]

[1]
[0.19943419]

[0]
[0.00362694]

[0]
[0.18895987]

[0]
[0.03527892]

[0]
[0.02889901]

[1]
[0.47615987]

[0]
[0.05549884]

[0]
[0.06553057]

[0]
[0.24727675]

[0]
[0.20818895]

[1]
[0.07698765]

[0]
[0.10154733]

[0]
[0.22964746]

[0]
[0.00940701]

[0]
[0.01500952]

[0]
[0.132613]

[1]
[0.8029233]

[0]
[0.05839932]

[0]
[0.05109945]

[0]
[0.2688015]

[1

In [560]:
predictions

array([[0.9666766 ],
       [0.0489383 ],
       [0.225043  ],
       ...,
       [0.0815953 ],
       [0.02645993],
       [0.09889734]], dtype=float32)

Binarisation de l'output décimal du modèle :

In [561]:
pred_bin = predictions.copy()
pred_bin[pred_bin<0.5] = 0
pred_bin[pred_bin>0.5] = 1
pred_bin

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [562]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred_bin, y_test2)

array([[1479,  214],
       [  93,  214]], dtype=int64)

In [563]:
model.metrics_names

['loss', 'accuracy']

In [564]:
accuracy = model.evaluate(x_test2,y_test2)
print("loss =", accuracy[0])
print("accuracy =", accuracy[1])

63/63 [==============================] - 0s 2ms/step - loss: 0.3788 - accuracy: 0.8465
loss = 0.378769189119339
accuracy = 0.8464999794960022


Le modèle a une accuracy passable et il prédit bien les négatifs mais il prédit trop de faux positifs.

En jouant sur le seuil de la binarisation des labels prédits, on peut essayer d'améliorer vaguement les choses...

In [565]:
pred_bin = predictions.copy()
pred_bin[pred_bin<0.4] = 0
pred_bin[pred_bin>0.4] = 1
pred_bin

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [566]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pred_bin, y_test2)

array([[1429,  191],
       [ 143,  237]], dtype=int64)

En regardant les données on voit qu'elles ne sont pas équilibrées et qu'il manque peut-être de matière pour entraîner les modèles sur les négatifs :

In [567]:
unique, counts = np.unique(y_train2, return_counts=True)
dict(zip(unique, counts))

{0: 6391, 1: 1609}

In [549]:
unique, counts = np.unique(y_test2, return_counts=True)
dict(zip(unique, counts))

{0: 1572, 1: 428}